In [59]:
import psycopg2
from pprint import pprint
import db_conn
from db_conn import *
import csv
import json
import pandas as pd

class DBConnection:
    def __init__(self,infile):
        try:
            params = config(infile)
            self.connection = psycopg2.connect(**params)
            self.connection.autocommit = True
            self.cursor = self.connection.cursor()
        except:
            pprint("Cannot connect to database")
            
    def execute_command(self,command):
        try:
            self.cursor.execute(command)
        except:
            print 'Error！！！'+command
    
    def find_attributes(self,table_name):
        self.cursor.execute("Select * FROM {0}".format(table_name))
        colnames = [desc[0] for desc in self.cursor.description]
        return colnames
    
    def insert_new_record(self,table_name,new_record,fill_attributes):#row by row insertion
        #new_record = table_attributes
        i = 0
        try:
            record = "INSERT INTO {0}".format(table_name) + fill_attributes + " VALUES('" 
            for j in range(0,len(new_record)-1):
                bug = new_record[j].replace('\'','\'\'').strip()
                record = record + bug + "','"
            bug2 = new_record[-1].replace('\'','\'\'').strip()
            record = record + bug2 + "')"
            insert_command = record.replace("'null'","null")
            #pprint(insert_command)
            #print insert_command
            i+=1
            try:
                self.cursor.execute(insert_command)
            except:
                print 'Error！！！'+insert_command
        except:
            print i
        
    def insert_new_record_lowercase(self,table_name,new_record,fill_attributes):#row by row insertion
        #new_record = table_attributes
        record = "INSERT INTO {0}".format(table_name) + fill_attributes + " VALUES('" 
        for j in range(0,len(new_record)-1):
            bug = new_record[j].replace('\'','\'\'').strip()
            record = record + bug + "','"
        bug2 = new_record[-1].replace('\'','\'\'').strip()
        record = record + bug2 + "')"
        insert_command = record.replace("'null'","null").lower()
        #pprint(insert_command)
        #print insert_command
        try:
            self.cursor.execute(insert_command)
        except:
            print 'Error！！！'+insert_command

    def query_all(self,table_name):
        self.cursor.execute("SELECT * FROM {0}".format(table_name))
        tuple_all = self.cursor.fetchall()
        return tuple_all
        #for tuple_each in tuple_all:
            #pprint("each_tuple : {0}".format(tuple_each))

    def drop_table(self,table_name):
        drop_table_command = "DROP TABLE {0}".format(table_name)
        self.cursor.execute(drop_table_command)


def gen_realtion_attribute(fieldnames):
    fill_attributes = '('
    for i in range(0,len(fieldnames)-1):
        fill_attributes += fieldnames[i]+','
    fill_attributes = fill_attributes + fieldnames[-1] + ')'
    return fill_attributes


def fetch_database_data(infile, command, *values):
    params = config(infile)
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    if not values:        
        try:
            cur.execute(command)      
            rows = cur.fetchall()
            return rows
        except psycopg2.Error as e:
            print e
    else:
        try:
            cur.execute(command, values)      
            rows = cur.fetchall()
            return rows
        except psycopg2.Error as e:
            print e        
    cur.close()
    conn.close()


In [39]:
def insert_data_from_aws(table_name_from,table_name_to,field_names,database_from):
    try:
        tuple_all = fetch_database_data(database_from, "Select * from {0};".format(table_name_from))
    except:
        print 'no such table in aws_old'
    fill_attributes = gen_realtion_attribute(field_names)    
    for tuple_each_in in range(0,len(tuple_all)):
        tuple_each = list(tuple_all[(tuple_each_in)])
        for each_item_index in range(0,len(tuple_each)):
            try:
                tuple_each[each_item_index] = str(int(tuple_each[each_item_index])).replace("nan","null").replace("None","null")
            except:
                tuple_each[each_item_index] = str(tuple_each[each_item_index]).replace("nan","null").replace("None","null")
            #print tuple_each[each_item_index]
            #tuple_each[each_item_index].replace("nan","null")
        #print tuple_each
        database_connection.insert_new_record(table_name_to,tuple_each,fill_attributes)


In [40]:
def insert_data_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,database_from):
    try:
        tuple_all = fetch_database_data(database_from, "Select * from {0};".format(table_name_from))
    except:
        print 'no such table in aws_old'
    database_connection2 = DBConnection(database_from)
    column_names = database_connection2.find_attributes(table_name_from)
    extract_index = []
    #print column_names
    for field_name_each in field_names_from:
        extract_index.append(column_names.index(field_name_each.lower().strip()))
    fill_attributes = gen_realtion_attribute(field_names_to)    
    for tuple_each_in in range(0,len(tuple_all)):
        tuple_each_new = []
        tuple_each = list(tuple_all[(tuple_each_in)])
        for each_item_index in range(0,len(extract_index)):
            try:
                tuple_each[extract_index[each_item_index]] = str(int(tuple_each[extract_index[each_item_index]])).replace("nan","null").replace("None","null")
            except:
                tuple_each[extract_index[each_item_index]] = str(tuple_each[extract_index[each_item_index]]).replace("nan","null").replace("None","null")
            tuple_each_new.append(tuple_each[extract_index[each_item_index]])
            #print tuple_each[each_item_index]
            #tuple_each[each_item_index].replace("nan","null")
        #print tuple_each_new
        database_connection.insert_new_record(table_name_to,tuple_each_new,fill_attributes)


In [41]:
def company_index_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,source_id,database_from):
    try:
        tuple_all = fetch_database_data(database_from, "Select * from {0};".format(table_name_from))
    except:
        print 'no such table in aws_old'
    database_connection2 = DBConnection(database_from)
    column_names = database_connection2.find_attributes(table_name_from)
    extract_index = []
    #print column_names
    for field_name_each in field_names_from:
        extract_index.append(column_names.index(field_name_each.lower().strip()))
    fill_attributes = gen_realtion_attribute(field_names_to)    
    for tuple_each_in in range(0,len(tuple_all)):
        tuple_each_new = []
        tuple_each = list(tuple_all[(tuple_each_in)])
        for each_item_index in range(0,len(extract_index)):
            try:
                tuple_each[extract_index[each_item_index]] = str(int(tuple_each[extract_index[each_item_index]])).replace("nan","null").replace("None","null")
            except:
                tuple_each[extract_index[each_item_index]] = str(tuple_each[extract_index[each_item_index]]).replace("nan","null").replace("None","null")
            tuple_each_new.append(tuple_each[extract_index[each_item_index]])
            for i in range(0,len(source_id)):
                tuple_each_new.append(source_id[i])
            #print tuple_each[each_item_index]
            #tuple_each[each_item_index].replace("nan","null")
        #print tuple_each_new
        database_connection.insert_new_record(table_name_to,tuple_each_new,fill_attributes)


In [42]:
def import_existing_table(table_name,filed_name,source_id,csv_path,extract_attributes):
    fill_attributes = gen_realtion_attribute(fieldnames)              
    tuple_list  = []
    for i in range(0,len(extract_attributes)):
        tuple_list.append([])
    attribute_index = []
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        for aim_attribute in extract_attributes:
            index = tuple_all[0].index(aim_attribute)
            attribute_index.append(index)
        del tuple_all[0]
    for idx in range(0,len(tuple_all)):
        for j in range(0,len(extract_attributes)):
            tuple_list[j].append(tuple_all[idx][attribute_index[j]])
    tuple_new = zip(*tuple_list)
    count_not_null = 0
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        flag = 1
        for i in range(0,len(tuple_each)):
            if len(tuple_each[i].strip()) == 0 or tuple_each[i].lower() == 'none':
                tuple_each[i] = 'null'          
        if flag == 1:
            for source_index in range(0,len(source_id)):
                tuple_each.append(source_id[source_index])
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [43]:
def insert_company_attribute(table_name,filed_name,source_id,csv_path,aim_attribute):
    fill_attributes = gen_realtion_attribute(fieldnames)              
    tuple_list = []
    for i in range(0,len(filed_name)-1):
        tuple_list.append([])
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        index = tuple_all[0].index(aim_attribute)
        tuple_all.pop(0)
    for company_idx in range(0,len(tuple_all)):
        tuple_list[0].append(tuple_all[company_idx][0])
        tuple_list[1].append(tuple_all[company_idx][index])
        tuple_list[2].append(str(source_id[0]))
    tuple_new = zip(*tuple_list)
    count_not_null = 0
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        flag = 1
        for i in range(0,len(tuple_each)):
            if len(tuple_each[i].strip()) == 0:
                tuple_each[i] = 'null'
                flag = 0
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
            count_not_null += 1
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [44]:
def insert_company_attribute_size(table_name,filed_name,source_id,csv_path,aim_attribute):
    fill_attributes = gen_realtion_attribute(fieldnames)              
    tuple_list = []
    for i in range(0,len(filed_name)-1):
        tuple_list.append([])
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        index = tuple_all[0].index(aim_attribute)
        tuple_all.pop(0)
    for company_idx in range(0,len(tuple_all)):
        tuple_list[0].append(tuple_all[company_idx][0])
        tuple_list[1].append(tuple_all[company_idx][index].replace(',',''))
        #print type(tuple_all[company_idx][index])
        tuple_list[2].append(str(source_id[0]))
    tuple_new = zip(*tuple_list)
    count_not_null = 0
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        flag = 1
        for i in range(0,len(tuple_each)):
            if len(tuple_each[i].strip()) == 0:
                tuple_each[i] = 'null'
                flag = 0
        
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
            count_not_null += 1
            #print tuple_each
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [45]:
def insert_company_attribute_lower(table_name,filed_name,source_id,csv_path,aim_attribute):
    fill_attributes = gen_realtion_attribute(fieldnames)              
    tuple_list = []
    for i in range(0,len(filed_name)-1):
        tuple_list.append([])
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        index = tuple_all[0].index(aim_attribute)
        tuple_all.pop(0)
        tuple_all.pop(0)
    for company_idx in range(0,len(tuple_all)):
        tuple_list[0].append(tuple_all[company_idx][0])
        tuple_list[1].append(tuple_all[company_idx][index])
        tuple_list[2].append(str(source_id[0]))
    tuple_new = zip(*tuple_list)
    count_not_null = 0
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        flag = 1
        for i in range(0,len(tuple_each)):
            if len(tuple_each[i].strip()) == 0:
                tuple_each[i] = 'null'
                flag = 0
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
            count_not_null += 1
            database_connection.insert_new_record_lowercase(table_name,tuple_each,fill_attributes)

In [46]:
def insert_company_OnetoM(table_name,filed_name,source_id,csv_path,extract_attributes):
    fill_attributes = gen_realtion_attribute(fieldnames)              
    tuple_list = []
    for i in range(0,len(filed_name)-1):
        tuple_list.append([])
    attribute_index = []
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        for aim_attribute in extract_attributes:
            index = tuple_all[0].index(aim_attribute)
            attribute_index.append(index)
        del tuple_all[0]
        del tuple_all[0]
    for company_idx in range(0,len(tuple_all)):
        for j in range(0,len(attribute_index)):
            tuple_list[0].append(tuple_all[company_idx][0])
            tuple_list[1].append(tuple_all[company_idx][attribute_index[j]])
            tuple_list[2].append(str(source_id[j]))
    tuple_new = zip(*tuple_list)
    count_not_null = 0
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        flag = 1
        for i in range(0,len(tuple_each)):
            if len(tuple_each[i].strip()) == 0 or tuple_each[i].lower() == 'none':
                tuple_each[i] = 'null'
                flag = 0   
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
            count_not_null += 1
            #print tuple_each
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [47]:
def company_url_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,source_id_dict,database_from):
    try:
        tuple_all = fetch_database_data(database_from, "Select * from {0};".format(table_name_from))
    except:
        print 'no such table in aws_old'
    database_connection2 = DBConnection(database_from)
    column_names = database_connection2.find_attributes(table_name_from)
    extract_index = []
    #print column_names
    for field_name_each in field_names_from:
        extract_index.append(column_names.index(field_name_each.lower().strip()))
    fill_attributes = gen_realtion_attribute(field_names_to)    
    for tuple_each_in in range(0,len(tuple_all)):
        tuple_each_new = []
        tuple_each = list(tuple_all[(tuple_each_in)])
        for each_item_index in range(0,len(extract_index)-1):
            try:
                tuple_each[extract_index[each_item_index]] = str(int(tuple_each[extract_index[each_item_index]])).replace("nan","null").replace("None","null")
            except:
                tuple_each[extract_index[each_item_index]] = str(tuple_each[extract_index[each_item_index]]).replace("nan","null").replace("None","null")
            tuple_each_new.append(tuple_each[extract_index[each_item_index]])
        url_type = tuple_each[extract_index[-1]]
        aim_source_id =  (source_id_dict[url_type])
        tuple_each_new.append(aim_source_id)
                #print tuple_each_new
        database_connection.insert_new_record(table_name_to,tuple_each_new,fill_attributes)
         

In [48]:
def company_iot_url_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,source_id_dict,database_from):
    try:
        tuple_all = fetch_database_data(database_from, "Select * from {0};".format(table_name_from))
    except:
        print 'no such table in aws_old'
    database_connection2 = DBConnection(database_from)
    column_names = database_connection2.find_attributes(table_name_from)
    extract_index = []
    #print column_names
    for field_name_each in field_names_from:
        extract_index.append(column_names.index(field_name_each.lower().strip()))
    fill_attributes = gen_realtion_attribute(field_names_to)   
    i = 0
    for tuple_each_in in range(0,len(tuple_all)):
        tuple_each_new = []
        tuple_each = list(tuple_all[(tuple_each_in)])
        for each_item_index in range(0,len(extract_index)-1):
            try:
                tuple_each[extract_index[each_item_index]] = str(int(tuple_each[extract_index[each_item_index]])).replace("nan","null").replace("None","null")
            except:
                tuple_each[extract_index[each_item_index]] = str(tuple_each[extract_index[each_item_index]]).replace("nan","null").replace("None","null")
            tuple_each_new.append(tuple_each[extract_index[each_item_index]])
        url_type = tuple_each[extract_index[-1]]
        if  url_type == 'IoT_URL':
            aim_source_id =  (source_id_dict[url_type])
            tuple_each_new.insert(0,str(i))
            tuple_each_new.append(aim_source_id)
            #print tuple_each_new
            try:
                database_connection.insert_new_record(table_name_to,tuple_each_new,fill_attributes)
                i += 1
            except:
                print 'error'
   

In [49]:
def insert_address_M(table_name,filed_name,csv_path,extract_attributes):
    fill_attributes = gen_realtion_attribute(fieldnames) 
    tuple_address_all = []
    tuple_index = []
    tuple_list = []
    for i in range(0,len(extract_attributes)):
        tuple_list.append([])
    attribute_index = []
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        for aim_attribute in extract_attributes:
            index = tuple_all[0].index(aim_attribute)
            attribute_index.append(index)
        del tuple_all[0]
        del tuple_all[0]
    for company_idx in range(0,len(tuple_all)):
        if tuple_all[company_idx][attribute_index[-1]] not in tuple_list[-1]:
            for j in range(0,len(attribute_index)):
                tuple_list[j].append(tuple_all[company_idx][attribute_index[j]])   
    count_not_null = 0#the unique index the pimary key
    tuple_new = zip(*tuple_list)
    #################for address_table pick the valid tuple###################
    for tuple_each_in in range(0,len(tuple_new)):
        #print tuple_each_in
        tuple_each = list(tuple_new[tuple_each_in])
        #print tuple_each
        check_all_null = 0
        flag = 1
        for i in range(0,len(tuple_each)):
            if len(tuple_each[i].strip()) == 0 or tuple_each[i].strip().lower() == 'none' or tuple_each[i].strip().lower() == 'n/a': #:
                tuple_each[i] = 'null'
                check_all_null+=1
        if check_all_null == len(attribute_index):
                flag = 0
        #print flag
    ###############for company_address_table,pick the valid####################
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
           # full_address_each = tuple_each.pop(-1)
            full_address_each = tuple_each[-1]
            count_not_null += 1
            tuple_index.append(tuple_each[0])
            tuple_address_all.append(full_address_each)
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)
    return tuple_index,tuple_address_all

In [50]:
def insert_company_address(table_name,filed_name,source_id,csv_path,extract_attributes,company_address_type):
    fill_attributes = gen_realtion_attribute(fieldnames)              
    tuple_list = []
    for i in range(0,len(filed_name)-1):
        tuple_list.append([])
    attribute_index = []
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        for aim_attribute in extract_attributes:
            index = tuple_all[0].index(aim_attribute)
            attribute_index.append(index)
        del tuple_all[0]
    for company_idx in range(0,len(tuple_all)):
        tuple_list[0].append(tuple_all[company_idx][0])
        aim = tuple_all[company_idx][attribute_index[-1]]
        try:
            a = (company_dict[aim])
            tuple_list[1].append(a)
        except:
            tuple_list[1].append('null')
        tuple_list[2].append(company_address_type[0])
        tuple_list[-1].append(str(source_id[0]))
    tuple_new = zip(*tuple_list)
   ###########################this is filter function##########################################
    count_not_null = 0
    flag = 1
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        flag = 1
        for i in range(0,len(tuple_each)):#BECAUSE THIS FLAG IS NOT THE FIRST
            if len(tuple_each[i].strip()) == 0 or tuple_each[i].strip().lower() == 'none' or tuple_each[i].strip().lower() == 'n/a' or  tuple_each[i].strip().lower() == 'null':
                tuple_each[i] = 'null'
                flag = 0
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
            count_not_null += 1
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [51]:
def insert_title_attribute(table_name,filed_name,source_id,csv_path,aim_attribute):
    #table_name = "company_phone_table" 
    #fieldnames = ("cpt_id","company_id"," source_id","company_phone")
    ################################################################################################################
    fill_attributes = gen_realtion_attribute(fieldnames)              
    print fill_attributes
    tuple_list = []
    for i in range(0,len(filed_name)-1):
        tuple_list.append([])
    title_list0 = []
    title_list1 = []
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        index = tuple_all[0].index(aim_attribute)
        tuple_all.pop(0)
    #print len(tuple_all)
    for company_idx in range(0,len(tuple_all)):
        if tuple_all[company_idx][index] not in tuple_list[0]:
            #tuple_2.append(str(source_id))
            tuple_list[0].append(tuple_all[company_idx][index])
            tuple_list[1].append('null')
            tuple_list[-1].append(str(source_id[0]))
    tuple_new = zip(*tuple_list)
    #print tuple_new
    count_not_null = 0
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        flag = 1
        for i in range(0,len(tuple_each)):
            if len(tuple_each[i].strip()) == 0 or tuple_each[i].lower() == 'none' or tuple_each[i].lower() == 'n/a':
                tuple_each[i] = 'null'
        if tuple_each[0] == 'null' and tuple_each[1] == 'null':
            flag = 0
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
            title_list0.append(tuple_each[0])
            title_list1.append(tuple_each[1])
            count_not_null += 1
            #print tuple_each
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)
    return title_list0,title_list1

In [52]:
def insert_people_index(table_name,filed_name,source_id,csv_path,aim_attribute):
    fill_attributes = gen_realtion_attribute(fieldnames)              
    tuple_list = []
    for i in range(0,len(filed_name)):
        tuple_list.append([])
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        index = 0#tuple_all[0].index(aim_attribute)
        tuple_all.pop(0)
    for person_idx in range(0,len(tuple_all)):
        tuple_list[0].append(tuple_all[person_idx][index])
        tuple_list[1].append(str(source_id[0]))
        tuple_list[2].append(str(source_id[1]))
    tuple_new = zip(*tuple_list)
    count_not_null = 0
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        flag = 1
        for i in range(0,len(tuple_each)):
            if len(tuple_each[i].strip()) == 0 or tuple_each[i].lower() == 'none' or tuple_each[i].lower() == 'n/a':
                tuple_each[i] = 'null'
                flag = 0
#############################filter tuples#############################################           
        if flag == 1:
            #count_not_null += 1
            #print tuple_each
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [53]:
def insert_person_attribute(table_name,filed_name,source_id,csv_path,aim_attribute):
    fill_attributes = gen_realtion_attribute(fieldnames)              
    tuple_list = []
    for i in range(0,len(filed_name)-1):
        tuple_list.append([])
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        index = tuple_all[0].index(aim_attribute)
        tuple_all.pop(0)
    for company_idx in range(0,len(tuple_all)):
        tuple_list[0].append(tuple_all[company_idx][0])
        tuple_list[1].append(tuple_all[company_idx][index])
        tuple_list[2].append('work')
        tuple_list[-1].append(str(source_id[0]))
    tuple_new = zip(*tuple_list)
    count_not_null = 0
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        flag = 1
        if len(tuple_each[1].strip()) == 0 or tuple_each[1].strip().lower() == 'none' or tuple_each[1].strip().lower() == 'n/a':
            flag = 0
            tuple_each[2] = 'null'
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
            count_not_null += 1
            #print tuple_each
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [54]:
def insert_person_OnetoM(table_name,filed_name,source_id,csv_path,extract_attributes,type_id):
    #table_name = "company_phone_table" 
    #fieldnames = ("cpt_id","company_id"," source_id","company_phone")
    ################################################################################################################
    fill_attributes = gen_realtion_attribute(fieldnames)              
    #print fill_attributes
    tuple_list = []
    for i in range(0,len(filed_name)-1):
        tuple_list.append([])
    attribute_index = []
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        #print tuple_all[0]
        for aim_attribute in extract_attributes:
            index = tuple_all[0].index(aim_attribute)
            attribute_index.append(index)
        del tuple_all[0]
    for company_idx in range(0,len(tuple_all)):
        for j in range(0,len(attribute_index)):
            tuple_list[0].append(tuple_all[company_idx][0])
            tuple_list[1].append(tuple_all[company_idx][attribute_index[j]])
            tuple_list[2].append(str(type_id[j]))
            tuple_list[-1].append(str(source_id[j]))
    #tuple_new = zip(tuple_2,tuple_3,tuple_4,tuple_5)
    tuple_new = zip(*tuple_list)
    count_not_null = 0
    
    for tuple_each in tuple_new:
        flag = 1
        tuple_each = list(tuple_each)
        if len(tuple_each[1].strip()) == 0 or tuple_each[2].strip().lower() == 'none' or tuple_each[1].strip().lower() == 'n/a':
            flag = 0
            tuple_each[2] = 'null'       
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
            count_not_null += 1
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [55]:
def insert_person_name(table_name,filed_name,source_id,csv_path,extract_attributes):
    fill_attributes = gen_realtion_attribute(fieldnames)              
    tuple_list = []
    for i in range(0,len(filed_name)-1):
        tuple_list.append([])
    attribute_index = []
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)
        for aim_attribute in extract_attributes:
            index = tuple_all[0].index(aim_attribute)
            attribute_index.append(index)
        del tuple_all[0]
    for company_idx in range(0,len(tuple_all)):
        tuple_list[0].append(tuple_all[company_idx][0])
        tuple_list[1].append(tuple_all[company_idx][attribute_index[0]])
        tuple_list[2].append(tuple_all[company_idx][attribute_index[1]])
        tuple_list[3].append(tuple_all[company_idx][attribute_index[2]])
        tuple_list[-1].append(str(source_id[0]))
    tuple_new = zip(*tuple_list)
    count_not_null = 0
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        if len(tuple_each[1].strip()) == 0 and len(tuple_each[3].strip()) == 0:
            flag = 0
            tuple_each[1] = 'null'
            tuple_each[3] = 'null'
        else:
            flag = 1
        if len(tuple_each[2].strip()) == 0:
            tuple_each[2] = 'null'
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
            count_not_null += 1
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [56]:
def company_name_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,source_id_dict,keys,values,database_from):
    try:
        tuple_all = fetch_database_data(database_from, "Select * from {0};".format(table_name_from))
    except:
        print 'no such table in aws_old'
    database_connection2 = DBConnection(database_from)
    column_names = database_connection2.find_attributes(table_name_from)
    extract_index = []
    #print column_names
    for field_name_each in field_names_from:
        extract_index.append(column_names.index(field_name_each.lower().strip()))
    fill_attributes = gen_realtion_attribute(field_names_to)   
    list_keys = []
    list_values = []
    for tuple_each_in in range(0,len(tuple_all)):
        tuple_each_new = []
        tuple_each = list(tuple_all[(tuple_each_in)])
        for each_item_index in range(0,len(extract_index)-1):
            try:
                tuple_each[extract_index[each_item_index]] = str(int(tuple_each[extract_index[each_item_index]])).replace("nan","null").replace("None","null")
            except:
                tuple_each[extract_index[each_item_index]] = str(tuple_each[extract_index[each_item_index]]).replace("nan","null").replace("None","null")
            tuple_each_new.append(tuple_each[extract_index[each_item_index]])
        url_type = tuple_each[extract_index[-1]]
        aim_source_id =  (source_id_dict[url_type])
        tuple_each_new.append(aim_source_id)
        database_connection.insert_new_record(table_name_to,tuple_each_new,fill_attributes)
        list_keys.append(tuple_each_new[field_names_to.index(keys)])
        list_values.append(tuple_each_new[field_names_to.index(values)])
    return list_keys, list_values
            

In [57]:
def insert_person_job(table_name,filed_name,source_id,csv_path,extract_attributes,current):
    fill_attributes = gen_realtion_attribute(fieldnames)              
    #print fill_attributes
    tuple_list = []
    for i in range(0,len(filed_name)-1):
        tuple_list.append([])
    attribute_index = []
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        tuple_all= list(reader)       
        for aim_attribute in extract_attributes:
            index = tuple_all[0].index(aim_attribute)
            attribute_index.append(index)
        del tuple_all[0]
    for company_idx in range(0,len(tuple_all)):
        aim0 = tuple_all[company_idx][attribute_index[0]]
        try:
            a0 = company_dict[aim0]
            #print a
            tuple_list[0].append(a0)
            #print title_dict[aim]
        except:
            tuple_list[0].append('null') 
        tuple_list[1].append(tuple_all[company_idx][0])
        aim = tuple_all[company_idx][attribute_index[1]]
        try:
            a = title_dict[aim]
            tuple_list[2].append(a)
        except:
            tuple_list[2].append('null')
    
        tuple_list[3].append(tuple_all[company_idx][attribute_index[2]])
        tuple_list[4].append(tuple_all[company_idx][attribute_index[3]])
        tuple_list[5].append(current)
        tuple_list[6].append(source_id[0])
       
    tuple_new = zip(*tuple_list)
    count_not_null = 0
    flag = 1
    for tuple_each in tuple_new:
        tuple_each = list(tuple_each)
        flag = 1
        for i in range(0,3):#because this flag is not the last one
            if len(tuple_each[i].strip()) == 0 or tuple_each[i].strip().lower() == 'none' or tuple_each[i].strip().lower() == 'n/a' or  tuple_each[i].strip().lower() == 'null':
                tuple_each[i] = 'null'
                flag = 0
        if flag == 1:
            tuple_each.insert(0,str(count_not_null))
            count_not_null += 1
            #print tuple_each
            database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [21]:
database_connection = DBConnection('database.ini')
database_from = 'database_aws_old.ini'
table_name_from = "coe_table"
table_name_to = "coe_table"
field_names = ("coe_id","coe_name","coe_type","coe_year","coe_expiration_date","coe_expire")
insert_data_from_aws(table_name_from,table_name_to,field_names,database_from)

In [22]:
table_name_from = "coe_tag_table"
table_name_to = "coe_tag_table"
field_names = ("coe_tag_id","coe_tag_name","coeparent_tag_id")
insert_data_from_aws(table_name_from,table_name_to,field_names,database_from)

In [23]:
table_name_from = "coe_tag_combination_table" 
table_name_to = "coe_tag_combination_table" 
field_names = ("coe_tag_combination_id","coe_tag_id"," coe_id ")
insert_data_from_aws(table_name_from,table_name_to,field_names,database_from)

In [24]:
table_name_from = "url_table" 
table_name_to = "coe_url_table"
field_names_to = ("coe_url_id","coe_url","coe_html_path","time_stamp","coe_id","coe_expired","coe_extracted_csv_path","coe_extract_type")
field_names_from = ["url_id","url","html_path","timestamp","coe_id","expired","extracted_csv_path","extract_type"]
insert_data_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,database_from)

In [25]:
table_name_from = "membership_table" 
table_name_to = "coe_membership_table" 
field_names_from = ["membership_id","membership_name","membership_score","url_id"]
field_names_to = ("coe_membership_id","coe_membership_name","coe_membership_score","coe_url_id")
insert_data_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,database_from)

In [26]:
table_name = "user_table" 
fieldnames = ("user_id","user_name","user_type")
fill_attributes = gen_realtion_attribute(fieldnames)              
tuple_all = [['0','out_source',"outsource"],['1','data_team',"employee"],['2','data_team',"employee"],['3','data_team',"employee"],['4','data_team',"employee"]]
for tuple_each in tuple_all:
    for i in range(0,len(tuple_each)):
        if len(tuple_each[i].strip()) == 0:
                tuple_each[i] = 'null'
    database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [27]:
table_name = "manual_table" 
fieldnames = ("manual_id","user_id","time_stamp")
fill_attributes = gen_realtion_attribute(fieldnames)              
tuple_all = [['0','0','7/28/17 10:23:54'],['1','1','8/24/17 10:23:54'],['2','2','3/29/18 10:23:54'],['3','3','8/4/18 10:23:54'],['4','4','8/14/18 10:23:54']]
for tuple_each in tuple_all:
    for i in range(0,len(tuple_each)):
        if len(tuple_each[i].strip()) == 0:
                tuple_each[i] = 'null'
    database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [60]:
table_name_to = "auto_table" 
field_names_to = ("auto_id","coe_url_id","to_be_further_added")   
table_name_from = "url_table"
field_names_from = ['url_id']  
try:
    tuple_all = fetch_database_data(database_from, "Select * from {0};".format(table_name_from))
except:
    print 'no such table in aws_old'

database_connection2 = DBConnection(database_from)
column_names = database_connection2.find_attributes(table_name_from)
extract_index = []  

for field_name_each in field_names_from:
    extract_index.append(column_names.index(field_name_each.lower().strip()))
    
fill_attributes = gen_realtion_attribute(field_names_to) 
print fill_attributes
for tuple_each_in in range(0,len(tuple_all)):
    tuple_each_new = []
    tuple_each = list(tuple_all[(tuple_each_in)])
    for each_item_index in range(0,len(extract_index)):
        try:
            tuple_each[extract_index[each_item_index]] = str(int(tuple_each[extract_index[each_item_index]])).replace("nan","null").replace("None","null")
        except:
            tuple_each[extract_index[each_item_index]] = str(tuple_each[extract_index[each_item_index]]).replace("nan","null").replace("None","null")
        tuple_each_new.append(tuple_each[extract_index[each_item_index]])
    tuple_each_new.append('n/a')
    tuple_each_new.insert(0,str(tuple_each_in))
    #print tuple_each_new
    database_connection.insert_new_record(table_name_to,tuple_each_new,fill_attributes)
    
    
    

(auto_id,coe_url_id,to_be_further_added)


In [90]:

table_name = "source_table" 
fieldnames = ("source_id","auto_id","manual_id","source_confidence","source_description")
fill_attributes = gen_realtion_attribute(fieldnames)              
tuple_1_s = ['0','1','2','3','4']
tuple_2_s = ['','','','','']
tuple_3_s = ['0','1','2','3','4']
tuple_4_s = ['0','0','5','5','10']
tuple_5_s = ['outsource confidence 0','employee at time 1 confidence 0','employee at time 2 confidence 5','employee at time 3 confidence 10','employee at time 4 confidence 10']
len_manual = len(tuple_1_s)

for source_id in range(0,len(tuple_1_s)):
    tuple_1_s.append(str(source_id+len_manual))
    tuple_2_s.append(tuple_1[source_id])
    tuple_3_s.append('')
    tuple_4_s.append('1')
    tuple_5_s.append('n/a')

tuple_all_s = zip(tuple_1_s,tuple_2_s,tuple_3_s,tuple_4_s,tuple_5_s)
for tuple_each in tuple_all_s:
    tuple_each = list(tuple_each)
    for i in range(0,len(tuple_each)):
        if len(tuple_each[i].strip()) == 0:
            tuple_each[i] = 'null'
    database_connection.insert_new_record(table_name,tuple_each,fill_attributes)

In [91]:
table_name_from = "company_basic_information_table" 
table_name_to = "company_table"  
field_names_from = ["company_id"]
field_names_to = ("company_id","first_establish_id","last_update_id")
source_id = ['1','2']
company_index_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,source_id,database_from)

In [92]:
table_name_to = "coe_company_table" 
table_name_from = "combination_table"
field_names_to = ("coe_combination_id","company_id"," coe_membership_id")
field_names_from = ['combination_id','company_id','membership_id']
insert_data_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,database_from)

In [93]:
table_name = "Company_Raw_Tag_Table" 
fieldnames = ("Company_Raw_Tag_Name","CRTT_Id","Source_ID ")
csv_path = 'TagNameToTagID - TagNameToTagID.csv'
extract_attributes = ['Tag Name','Tag Index ID']
source_id = ['1']
import_existing_table(table_name,fieldnames,source_id,csv_path,extract_attributes)

In [94]:
table_name = "Company_Tag_Relation_Table" 
fieldnames = ("CTRT_ID","Company_ID","CRTT_ID","Source_ID ")
csv_path = 'CompanyIDToTagID - CompanyIDToTagID.csv'
extract_attributes = ['Company Tag Relation ID','Company ID','Tag ID']
source_id = ['1']## this should correspond with extract_attributes
import_existing_table(table_name,fieldnames,source_id,csv_path,extract_attributes)

In [95]:
table_name = "Polished_Tag_Table" 
fieldnames = ("PTT_Id","CRTT_ID","Ptt_Type","Source_ID ")
csv_path = 'TagIDWithVerticalHorizontal - TagIDWithVerticalHorizontal.csv'
extract_attributes = ['Tag Vertical Horizontal Relation ID','Tag ID','Vertical or Horizontal']
source_id = ['1']
import_existing_table(table_name,fieldnames,source_id,csv_path,extract_attributes)

In [96]:
table_name = "Raw_Tag_Polished_Tag_Relation_Table" 
fieldnames = ("RTPTRT_id","CRTT_ID","PTT_Id","Source_ID ")
csv_path = 'TagIDPolishedTagIDRelation - TagIDPolishedTagIDRelation.csv'
extract_attributes = ['Tag ID Polished Tag Relation ID Relation','Tag ID','Polished Tag Relation ID']
source_id = ['1']
import_existing_table(table_name,fieldnames,source_id,csv_path,extract_attributes)

In [97]:
table_name = "company_phone_table" 
fieldnames = ("cpt_id","company_id","company_phone"," source_id")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
aim_attribute = 'Company Phone'
source_id = ['1']
insert_company_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

Error！！！INSERT INTO company_phone_table(cpt_id,company_id,company_phone, source_id) VALUES('160','203','+32 479 44 58 58','1')
Error！！！INSERT INTO company_phone_table(cpt_id,company_id,company_phone, source_id) VALUES('530','712','+49 6452 91400','1')
Error！！！INSERT INTO company_phone_table(cpt_id,company_id,company_phone, source_id) VALUES('556','746','+1 650-503-2200','1')
Error！！！INSERT INTO company_phone_table(cpt_id,company_id,company_phone, source_id) VALUES('562','754','+86 10 6230 1618','1')
Error！！！INSERT INTO company_phone_table(cpt_id,company_id,company_phone, source_id) VALUES('694','940','+46 8 525 002 50','1')
Error！！！INSERT INTO company_phone_table(cpt_id,company_id,company_phone, source_id) VALUES('709','965','+48 12 646 10 00','1')
Error！！！INSERT INTO company_phone_table(cpt_id,company_id,company_phone, source_id) VALUES('714','971','+1 510-972-9709','1')
Error！！！INSERT INTO company_phone_table(cpt_id,company_id,company_phone, source_id) VALUES('748','1017','+44 1223 2

In [98]:
table_name = "company_email_table" 
fieldnames = ("cet_id","company_id","company_email"," source_id")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
aim_attribute = 'Company Email'
source_id = ['1']
insert_company_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

Error！！！INSERT INTO company_email_table(cet_id,company_id,company_email, source_id) VALUES('169','203','hello@allorafactory.com','1')
Error！！！INSERT INTO company_email_table(cet_id,company_id,company_email, source_id) VALUES('582','712','info@embedded-intelligence.de','1')
Error！！！INSERT INTO company_email_table(cet_id,company_id,company_email, source_id) VALUES('614','746','support@c3iot.com','1')
Error！！！INSERT INTO company_email_table(cet_id,company_id,company_email, source_id) VALUES('621','754','shenyaqin@catr.cn','1')
Error！！！INSERT INTO company_email_table(cet_id,company_id,company_email, source_id) VALUES('770','940','sales@clxcommunications.com','1')
Error！！！INSERT INTO company_email_table(cet_id,company_id,company_email, source_id) VALUES('792','965','contact@com4innov.com','1')
Error！！！INSERT INTO company_email_table(cet_id,company_id,company_email, source_id) VALUES('797','971','info@comfyapp.com','1')
Error！！！INSERT INTO company_email_table(cet_id,company_id,company_email,

In [99]:
table_name = "company_blog_table" 
fieldnames = ("cbt_id","company_id","company_blog"," source_id")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
aim_attribute = 'Blog'
source_id = ['1']
insert_company_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

Error！！！INSERT INTO company_blog_table(cbt_id,company_id,company_blog, source_id) VALUES('302','940','https://www.clxcommunications.com/blog/','1')
Error！！！INSERT INTO company_blog_table(cbt_id,company_id,company_blog, source_id) VALUES('310','971','https://www.comfyapp.com/blog/','1')
Error！！！INSERT INTO company_blog_table(cbt_id,company_id,company_blog, source_id) VALUES('418','1372','https://www.eitdigital.eu/newsroom/blog/','1')
Error！！！INSERT INTO company_blog_table(cbt_id,company_id,company_blog, source_id) VALUES('847','3050','https://nest.com/blog/','1')
Error！！！INSERT INTO company_blog_table(cbt_id,company_id,company_blog, source_id) VALUES('1094','3938','https://blog.seebo.com/','1')
Error！！！INSERT INTO company_blog_table(cbt_id,company_id,company_blog, source_id) VALUES('1096','3940','https://blog.silvair.com/','1')
Error！！！INSERT INTO company_blog_table(cbt_id,company_id,company_blog, source_id) VALUES('1416','5032','https://halosmartlabs.com/blogs/news','1')
Error！！！INSERT

In [100]:
table_name = "company_description_table" 
fieldnames = ("cdt_id","company_id","company_description"," source_id")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
aim_attribute = 'Company Description'
source_id = ['1']
insert_company_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

Error！！！INSERT INTO company_description_table(cdt_id,company_id,company_description, source_id) VALUES('142','754','China Academy of Telecommunication Research (CATR), with its predecessor being China Academy of Posts and Telecommunications of the former Ministry of Posts and Telecommunications (MPT) established in the mid-1950s, has since its founding undertaken a great deal of major  research for the country and the industry. It has thus accumulated rich experience in communications policies, standards for telecommunication technologies, planning of communications development, and product testing and certification, making great contributions to the development of communications industry in China.<br />　　Since the founding of the Ministry of Industry and Information Technology (MIIT), CATR, deepening reform and bringing integrated advantage into full play on the basis of development, has become the pillar unit in the ICT field in China and a major supportive unit for MIIT in comprehen

Error！！！INSERT INTO company_description_table(cdt_id,company_id,company_description, source_id) VALUES('1373','6281','In 2003 Skyhook Wireless developed an innovative way to obtain location. Using Wi-Fi signals would be faster, more precise and battery-friendly than GPS and cell-towers alone.  It collected the locations of billions of Wi-Fi access points worldwide and built the infrastructure and software to deliver the information.  Its products include Skyhook Precision Location, Skyhook Context, and Skyhook Personas for connectivity locations and insights into user data.','1')
Error！！！INSERT INTO company_description_table(cdt_id,company_id,company_description, source_id) VALUES('1390','6298','Esri inspires and enables people to positively impact their future through a deeper, geographic understanding of the changing world around them. ArcGIS provides contextual tools for mapping and spatial reasoning so you can explore data and share location-based insights. ArcGIS is a complete sys

In [101]:
table_name = "company_facebook_table" 
fieldnames = ("cft_id","company_id","company_facebook"," source_id")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
aim_attribute = 'Facebook'
source_id = ['1']
insert_company_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

Error！！！INSERT INTO company_facebook_table(cft_id,company_id,company_facebook, source_id) VALUES('136','203','https://www.facebook.com/alloraFactory/','1')
Error！！！INSERT INTO company_facebook_table(cft_id,company_id,company_facebook, source_id) VALUES('496','746','https://www.facebook.com/C3IoT/','1')
Error！！！INSERT INTO company_facebook_table(cft_id,company_id,company_facebook, source_id) VALUES('630','940','https://www.facebook.com/CLX-Networks-157221240998497/','1')
Error！！！INSERT INTO company_facebook_table(cft_id,company_id,company_facebook, source_id) VALUES('646','965','https://www.facebook.com/comarchpl','1')
Error！！！INSERT INTO company_facebook_table(cft_id,company_id,company_facebook, source_id) VALUES('650','971','https://www.facebook.com/comfyapp/','1')
Error！！！INSERT INTO company_facebook_table(cft_id,company_id,company_facebook, source_id) VALUES('721','1083','https://www.facebook.com/CttcTech','1')
Error！！！INSERT INTO company_facebook_table(cft_id,company_id,company_fac

In [102]:
table_name = "company_linkedin_table" 
fieldnames = ("clt_id","company_id","company_linkedin"," source_id",)
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
aim_attribute = 'Linkedin'
source_id = ['1']
insert_company_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

Error！！！INSERT INTO company_linkedin_table(clt_id,company_id,company_linkedin, source_id) VALUES('158','203','https://www.linkedin.com/company-beta/10398402/','1')
Error！！！INSERT INTO company_linkedin_table(clt_id,company_id,company_linkedin, source_id) VALUES('564','746','https://www.linkedin.com/company-beta/414478','1')
Error！！！INSERT INTO company_linkedin_table(clt_id,company_id,company_linkedin, source_id) VALUES('720','940','https://www.linkedin.com/company-beta/634222/','1')
Error！！！INSERT INTO company_linkedin_table(clt_id,company_id,company_linkedin, source_id) VALUES('740','965','https://www.linkedin.com/company-beta/3529/','1')
Error！！！INSERT INTO company_linkedin_table(clt_id,company_id,company_linkedin, source_id) VALUES('745','971','https://www.linkedin.com/company-beta/7968992/','1')
Error！！！INSERT INTO company_linkedin_table(clt_id,company_id,company_linkedin, source_id) VALUES('777','1017','https://www.linkedin.com/company-beta/40315/','1')
Error！！！INSERT INTO company_

In [103]:
table_name = "company_twitter_table" 
fieldnames = ("ctt_id","company_id","company_twitter"," source_id")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
aim_attribute = 'Twitter'
source_id = ['1']
insert_company_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

Error！！！INSERT INTO company_twitter_table(ctt_id,company_id,company_twitter, source_id) VALUES('143','203','https://twitter.com/allorafactory','1')
Error！！！INSERT INTO company_twitter_table(ctt_id,company_id,company_twitter, source_id) VALUES('534','746','https://twitter.com/C3IoT','1')
Error！！！INSERT INTO company_twitter_table(ctt_id,company_id,company_twitter, source_id) VALUES('682','940','https://twitter.com/CLXCommsAB','1')
Error！！！INSERT INTO company_twitter_table(ctt_id,company_id,company_twitter, source_id) VALUES('700','965','https://twitter.com/comarchgroup','1')
Error！！！INSERT INTO company_twitter_table(ctt_id,company_id,company_twitter, source_id) VALUES('704','971','https://twitter.com/comfyapp','1')
Error！！！INSERT INTO company_twitter_table(ctt_id,company_id,company_twitter, source_id) VALUES('733','1017','https://twitter.com/Cyanconnode','1')
Error！！！INSERT INTO company_twitter_table(ctt_id,company_id,company_twitter, source_id) VALUES('786','1083','https://twitter.com/C

In [104]:
table_name = "Company_News_Table" 
fieldnames = ("cnews_id","company_id","company_news"," source_id")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
aim_attribute = 'News'
source_id = ['1']
insert_company_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

Error！！！INSERT INTO Company_News_Table(cnews_id,company_id,company_news, source_id) VALUES('253','746','http://c3iot.com/iot-news/','1')
Error！！！INSERT INTO Company_News_Table(cnews_id,company_id,company_news, source_id) VALUES('317','940','https://www.clxcommunications.com/news-events/','1')
Error！！！INSERT INTO Company_News_Table(cnews_id,company_id,company_news, source_id) VALUES('325','965','http://www.comarch.com/press-center/news/','1')
Error！！！INSERT INTO Company_News_Table(cnews_id,company_id,company_news, source_id) VALUES('328','971','https://www.comfyapp.com/press/','1')
Error！！！INSERT INTO Company_News_Table(cnews_id,company_id,company_news, source_id) VALUES('450','1400','http://corporate.elisa.com/responsibility/news/','1')
Error！！！INSERT INTO Company_News_Table(cnews_id,company_id,company_news, source_id) VALUES('608','1937','http://www.harvardtechnology.com/company/company-updates/news/','1')
Error！！！INSERT INTO Company_News_Table(cnews_id,company_id,company_news, source

In [105]:

table_name = "Company_size_Table" 
fieldnames = ("csize_id","company_id","company_size"," source_id")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
aim_attribute = 'Company Size'
source_id = ['1']
insert_company_attribute_size(table_name,fieldnames,source_id,csv_path,aim_attribute)

In [106]:
table_name = "Company_Status_Table" 
fieldnames = ("cst_id","company_id","company_status","last_update_id")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
aim_attribute = 'Status'
source_id = ['1']
insert_company_attribute_lower(table_name,fieldnames,source_id,csv_path,aim_attribute)

Error！！！insert into company_status_table(cst_id,company_id,company_status,last_update_id) values('732','754','inactive','1')
Error！！！insert into company_status_table(cst_id,company_id,company_status,last_update_id) values('989','1017','active','1')
Error！！！insert into company_status_table(cst_id,company_id,company_status,last_update_id) values('1529','1574','active','1')
Error！！！insert into company_status_table(cst_id,company_id,company_status,last_update_id) values('1711','1762','inactive','1')
Error！！！insert into company_status_table(cst_id,company_id,company_status,last_update_id) values('2160','2228','active','1')
Error！！！insert into company_status_table(cst_id,company_id,company_status,last_update_id) values('2305','2376','inactive','1')
Error！！！insert into company_status_table(cst_id,company_id,company_status,last_update_id) values('2500','2578','active','1')
Error！！！insert into company_status_table(cst_id,company_id,company_status,last_update_id) values('2546','2625','active','1

In [107]:
company_url_type = ["Official URL","Main_URL","Alternative_URL","IoT_URL"]
source_id_list = ['3','2','1','4']
source_id_dict = dict(zip(company_url_type,source_id_list))
table_name_from = "Company_URL_Table" 
table_name_to = "Company_URL_Table" 
field_names_from = ["cut_id","company_id","company_url","company_url_type"]
field_names_to = ("cut_id","company_id","company_url","source_id")
company_url_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,source_id_dict,database_from)

Error！！！INSERT INTO Company_URL_Table(cut_id,company_id,company_url,source_id) VALUES('27598','6001','http://www.iotracked.com/','1')
Error！！！INSERT INTO Company_URL_Table(cut_id,company_id,company_url,source_id) VALUES('31879','3867','www.sanmina.com','1')
Error！！！INSERT INTO Company_URL_Table(cut_id,company_id,company_url,source_id) VALUES('44146','7194','http://www.netapp.com','1')
Error！！！INSERT INTO Company_URL_Table(cut_id,company_id,company_url,source_id) VALUES('46626','1056','http://www.cradlepoint.com','1')
Error！！！INSERT INTO Company_URL_Table(cut_id,company_id,company_url,source_id) VALUES('46628','15128','http://www.IoTMktg.com','1')
Error！！！INSERT INTO Company_URL_Table(cut_id,company_id,company_url,source_id) VALUES('51141','1524','http://www.eurotech.com/en/products/software+services/everyware+software+framework/what+esf+is','1')
Error！！！INSERT INTO Company_URL_Table(cut_id,company_id,company_url,source_id) VALUES('51142','3707','https://www.redhat.com/iot','1')
Error！！

In [108]:
company_url_type = ["IoT_URL"]
source_id_list = ['4']
source_id_dict = dict(zip(company_url_type,source_id_list))
table_name_from = "Company_URL_Table" 
table_name_to = "iot_URL_Table" 
field_names_from = ["company_id","company_url","company_url_type"]
field_names_to = ("iut_id","company_id","company_iot_url","source_id")
company_iot_url_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,source_id_dict,database_from)

In [109]:
table_name = "company_iot_nation_rank" 
fieldnames = ("rank_id","company_id","company_iot_rank","source_id")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
extract_attributes = ['IoT Nation Rank Q2','IoT Nation Rank Q1']
source_id = [3,2]## this should correspond with extract_attributes
insert_company_OnetoM(table_name,fieldnames,source_id,csv_path,extract_attributes)

Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('269','203','5003','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('949','712','4997','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('995','746','2387','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('1006','754','447','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('1261','940','1383','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('1294','965','1377','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('1303','971','5123','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('1367','1017','4966','2')
Error！！！INSERT INTO

Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('8455','6119','2872','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('8536','6166','2767','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('8566','6183','2706','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('8590','6197','2682','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('8661','6234','2609','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('8695','6252','2770','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('8743','6281','694','2')
Error！！！INSERT INTO company_iot_nation_rank(rank_id,company_id,company_iot_rank,source_id) VALUES('8776','6298','840','2')
Error！！！IN

In [110]:
table_name = "address_table" 
fieldnames = ("address_id","street_1","street_2","city","state","zipcode","country","longitude","latitude","full_address")
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
extract_attributes = ['Headquarters Address: Street 1','Headquarters Address: Street 2','Headquarters Address: City','Headquarters Address: State','Headquarters Address: Zip','Headquarters Address: Country','Headquarters Address: Longitude','Headquarters Address: Latitude','Headquarter Address']
list0,list1 = insert_address_M(table_name,fieldnames,csv_path,extract_attributes)
company_dict = dict(zip(list1,list0))

In [111]:
table_name = "company_address_table" 
fieldnames = ("cas_id","company_id","address_id","company_address_type","source_id",)
csv_path = 'iotnationbd_20180605 - iotnationbd(all).csv'
company_address_type = ['hq']
extract_attributes = ['Headquarter Address']
source_id = ['1']
insert_company_address(table_name,fieldnames,source_id,csv_path,extract_attributes,company_address_type)

Error！！！INSERT INTO company_address_table(cas_id,company_id,address_id,company_address_type,source_id) VALUES('196','203','194','hq','1')
Error！！！INSERT INTO company_address_table(cas_id,company_id,address_id,company_address_type,source_id) VALUES('689','712','657','hq','1')
Error！！！INSERT INTO company_address_table(cas_id,company_id,address_id,company_address_type,source_id) VALUES('721','746','686','hq','1')
Error！！！INSERT INTO company_address_table(cas_id,company_id,address_id,company_address_type,source_id) VALUES('729','754','693','hq','1')
Error！！！INSERT INTO company_address_table(cas_id,company_id,address_id,company_address_type,source_id) VALUES('909','940','850','hq','1')
Error！！！INSERT INTO company_address_table(cas_id,company_id,address_id,company_address_type,source_id) VALUES('932','965','870','hq','1')
Error！！！INSERT INTO company_address_table(cas_id,company_id,address_id,company_address_type,source_id) VALUES('938','971','876','hq','1')
Error！！！INSERT INTO company_addres

Error！！！INSERT INTO company_address_table(cas_id,company_id,address_id,company_address_type,source_id) VALUES('8942','10657','7998','hq','1')


In [112]:
table_name = "job_title_table" 
fieldnames = ("jtt_id","original_title","re_constructed_title","source_id",)#index, source 
csv_path = 'iotnationbdcontacts.csv'
aim_attribute = 'Actual Title'
source_id = ['1']
title_list0,title_list1 = insert_title_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

(ptt_id,original_title,re_constructed_title,source_id)


In [113]:
from itertools import izip
title_dict = dict(zip(title_list1,title_list0))
#print title_zip
#print title_dict
A = title_dict['EVP Sales and Strategy']
print A

187


In [114]:
table_name = "person_index_table" 
fieldnames = ("person_id","first_establish_id","last_update_id")
csv_path = 'iotnationbdcontacts.csv'
aim_attribute = 'Index'
source_id = [1,2]## this should correspond with extract_attributes
insert_people_index(table_name,fieldnames,source_id,csv_path,aim_attribute)

In [115]:
table_name = "person_linkedin_table" 
fieldnames = ("plt_id","person_id","person_linkedin"," source_id")
csv_path = 'iotnationbdcontacts.csv'
aim_attribute = 'Linkedin Profile'
source_id = ['1']
insert_company_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

In [116]:
table_name = "person_phone_table" 
fieldnames = ("ppt_id ","person_id","person_phone_number","person_phone_type"," source_id")
csv_path = 'iotnationbdcontacts.csv'
aim_attribute = 'Contact Phone'
source_id = ['1']
insert_person_attribute(table_name,fieldnames,source_id,csv_path,aim_attribute)

In [117]:
table_name = "person_email_table" 
fieldnames = ("pet_id","person_id","person_email_address","person_email_type","source_id")
csv_path = 'iotnationbdcontacts.csv'
extract_attributes = ['Work Email','Email Address']
type_id = ['work','personal']
source_id = [3,4]
insert_person_OnetoM(table_name,fieldnames,source_id,csv_path,extract_attributes,type_id)

In [118]:
table_name = "person_name_table" 
fieldnames = ("pnt_id","person_id","person_first_name","person_middle_name","person_last_name","source_id")
csv_path = 'iotnationbdcontacts.csv'
extract_attributes = ['Full Name (Main): First','Full Name (Main): Middle','Full Name (Main): Last']
type_id = ['work','personal']
source_id = ['1']
insert_person_name(table_name,fieldnames,source_id,csv_path,extract_attributes)

In [119]:
company_name_type = ["Official Name","Main_Name","Known_As","Prev_Name","Other_Name"]
source_id_list = ['4','3','2','0','1']
source_id_dict = dict(zip(company_name_type,source_id_list))
table_name_from = "Company_Name_Table" 
table_name_to = "Company_Name_Table" 
field_names_from = ["cnt_id","company_id","company_name","company_name_type"]
field_names_to = ("cnt_id","company_id","company_name","source_id")
keys = 'company_name'
values = 'company_id'
list_keys,list_values = company_name_from_aws_inconsistent(table_name_from,table_name_to,field_names_from,field_names_to,source_id_dict,keys,values,database_from)

Error！！！INSERT INTO Company_Name_Table(cnt_id,company_id,company_name,source_id) VALUES('48209','5860','ADRAL Alentejo Regional Development Agency','1')
Error！！！INSERT INTO Company_Name_Table(cnt_id,company_id,company_name,source_id) VALUES('228957','1656','Flex','1')
Error！！！INSERT INTO Company_Name_Table(cnt_id,company_id,company_name,source_id) VALUES('69611','3171','Nippon Telegraph and Telephone Corporation','1')
Error！！！INSERT INTO Company_Name_Table(cnt_id,company_id,company_name,source_id) VALUES('228667','5334','Samsung ARTIK Cloud Services-SmartThings','1')
Error！！！INSERT INTO Company_Name_Table(cnt_id,company_id,company_name,source_id) VALUES('229115','3788','Robert Bosch GmbH','1')
Error！！！INSERT INTO Company_Name_Table(cnt_id,company_id,company_name,source_id) VALUES('229324','2349','Itron','1')
Error！！！INSERT INTO Company_Name_Table(cnt_id,company_id,company_name,source_id) VALUES('214053','229','AMAZON','1')
Error！！！INSERT INTO Company_Name_Table(cnt_id,company_id,compan

In [120]:
company_dict = dict(zip(list_keys,list_values))

In [121]:
table_name = "Person_Job_Table" 
fieldnames = ("PJT_ID","Company_ID","Person_ID"," jtt_id ","city","country","current","source_id")
#for_match = 'Headquarter Address'#,"Headquarter Address")
csv_path = 'iotnationbdcontacts.csv'
extract_attributes = ['Company Name','Actual Title','Location','Country','Company Name']
current = 'false'#default value
source_id = ['1']
insert_person_job(table_name,fieldnames,source_id,csv_path,extract_attributes,current)